In [192]:
from cmlreaders import CMLReader, get_data_index
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats

In [236]:
sites = ['Dartmouth', 'Jefferson']
sel_sub = pd.DataFrame([])
subs = []
for site in sites:
    df = pd.read_excel(r'./' + site + '_neuropsych.xlsx')
    for i in df.index:
        gap = df.iloc[i]['Time between report and implant (days)']
        BDI = df.iloc[i]['BDI']
        BAI = df.iloc[i]['BAI']
        sub = df.iloc[i]['Subject Number']
        if gap < 183 and np.isnan(BDI)==False and np.isnan(BAI)==False:
            sel_sub = sel_sub.append({
                'subject':sub,
                'BDI':BDI,
                'BAI':BAI
                }, ignore_index = True)
sel_sub

BAI   BDI  subject
0    3.0   6.0     32.0
1    9.0   8.0    116.0
2    3.0  13.0    317.0
3   35.0   8.0    330.0
4   10.0  10.0    347.0
5   29.0  26.0    367.0
6   10.0   1.0    376.0
7   24.0  39.0    413.0
8   15.0  36.0    450.0
9    4.0  25.0    456.0
10  19.0  38.0    494.0
11   0.0   2.0      8.0
12  14.0  22.0     19.0
13   3.0   1.0     22.0
14   7.0   6.0     49.0
15  20.0  30.0     51.0
16  17.0   5.0     55.0
17  11.0  13.0     59.0
18   9.0  21.0     73.0
19   0.0  11.0     81.0
20   9.0  19.0     83.0
21  20.0   6.0     93.0
22   1.0   6.0    107.0
23   1.0   1.0    108.0
24  20.0  13.0    117.0
25   4.0   5.0    145.0
26  15.0  18.0    150.0
27  10.0  16.0    241.0
28   4.0  41.0    266.0
29   4.0   1.0    267.0
30   5.0  16.0    286.0
31   0.0   1.0    343.0
32  10.0  11.0    384.0
33  14.0  29.0    389.0
34   6.0   5.0    398.0
35   4.0   4.0    401.0
36  13.0  14.0    436.0
37   2.0   4.0    482.0
38  13.0  10.0    505.0
39   6.0   3.0    525.0
40   5.0   6.0    526.0
41  18.0  16.0    530.0
42  24.0  17.0    536.0
43   5.0   5.0    564.0

In [241]:
df = get_data_index('r1')
df_subs = df['subject'].unique()
df_select = pd.DataFrame([])
df_subs
tmp = []
select = []
for sub in df_subs:
    num = int(sub[2:5])
    for i in sel_sub.index:
        sel = sel_sub.iloc[i]['subject']
        if sel == num:
            df_select = df_select.append({
                'subject':sub,
                'BDI':sel_sub.iloc[i]['BDI'],
                'BAI':sel_sub.iloc[i]['BAI']
                }, ignore_index = True)

In [246]:
trial_df = pd.DataFrame([])
experiments = ['FR1','catFR1']

for sub in df_select['subject']: ### 'sub' instead of 's' 
    try:
        for exp in experiments:
            localizations=df[(df['subject']==sub) & (df['experiment']==exp)]['localization'].unique()        

            for loc in localizations:
                montages = df[(df['subject']==sub) & (df['experiment']==exp) & (df['localization']==loc)]['montage'].unique()            

                for mont in montages: # iterate over montages            
                    data = df[np.logical_and(df["subject"] == sub, df['experiment']==exp)]
                    sessions=data[np.logical_and(data['montage']==mont,data['localization']==loc)]['session'].unique()

                    for sess in sessions: # 'sess' instead of 'k' 
                        reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)   #reader for loading CML data
                        evs = reader.load('task_events')
                        word_evs = evs[evs['type']=='REC_WORD']
    #                     rec_evs = word_evs[word_evs['recalled']==True]
                        s_dict = {} #create variables for each field and then append using Noa's technique

                        ### many times there are negative lists, referring to practice lists. You want to
                        # exclude those: word_evs['list'][word_evs['list']>0].unique() 
                        lists = len(word_evs['list'][word_evs['list']>0].unique()) 
                        corr_recs = len(word_evs[word_evs['intrusion']==0]) # changed
                        ELIs = len(word_evs[word_evs['intrusion']==-1])
                        PLIs = len(word_evs[word_evs['intrusion']>0])
                        total_ints = ELIs + PLIs
                        total_trials = corr_recs + total_ints

                        trials = []
                        for list_num in word_evs['list'][word_evs['list']>0].unique():
                            list_events=word_evs[word_evs['list']==list_num]
                            if len(list_events)>0: 
                                for i in range(0,len(list_events)):
                                    trials.append(list_events.index[i])               

                        trial_df = trial_df.append({
                            'subject':sub,
                            'session': sess,
                            'montage': mont,
                            'localization': loc,
                            'experiment': exp,
                            'number of lists': lists,
                            'number of correct recalls': corr_recs,
                            'number of extra-list intrusions': ELIs,
                            'number of prior-list intrusions': PLIs,
                            'total number of intrusions': total_ints,
                            'total number of trials left': total_trials,
                            'trials for analysis': trials
                            },ignore_index=True )  
    except:
        continue

In [248]:
df_filter = pd.DataFrame([])

for i in trial_df.index:
    try:
        sub = trial_df.iloc[i]['subject']
        row = df_select[df_select['subject'] == sub]
        eli = trial_df.iloc[i]['number of extra-list intrusions']
        pli = trial_df.iloc[i]['number of prior-list intrusions']
        correc = trial_df.iloc[i]['number of correct recalls']
        lists = trial_df.iloc[i]['number of lists']
        exp = trial_df.iloc[i]['experiment']
        #max number of lists is either 12 or 25 for catFR1 depending on version. Always 25 for FR1
        sc_correc = float(correc/lists)*25
        sceli = float(eli/lists)*25
        scpli = float(pli/lists)*25
        df_filter = df_filter.append({
            'subject':sub,
            'experiment':exp,
            'correct recalls':correc,
            'ELIs':eli,
            'PLIs':pli,
            'total ints': eli+pli,
            'lists':lists,
            'correct recalls scaled':sc_correc,
            'ELI scaled':sceli,
            'PLI scaled':scpli,
            'total scaled':sceli+scpli,
            'BAI':int(row['BAI'].values),
            'BDI':int(row['BDI'].values)
        }, ignore_index = True)
    except:
        continue
df_filter

BAI   BDI  ELI scaled  ELIs  PLI scaled  PLIs  correct recalls  \
0     3.0   1.0   25.000000  25.0   14.000000  14.0             83.0   
1     3.0   6.0    3.125000   3.0    7.291667   7.0             95.0   
2     3.0   6.0    4.166667   2.0    4.166667   2.0             36.0   
3     3.0   6.0    2.777778   2.0    1.388889   1.0             67.0   
4     7.0   6.0   36.458333  35.0   20.833333  20.0             27.0   
..    ...   ...         ...   ...         ...   ...              ...   
110  18.0  16.0    6.250000   3.0   52.083333  25.0            124.0   
111  24.0  17.0    4.545455   2.0    9.090909   4.0             46.0   
112  24.0  17.0   12.500000   6.0   18.750000   9.0             45.0   
113  24.0  17.0    8.333333   4.0   35.416667  17.0             50.0   
114  24.0  17.0    0.000000   0.0   31.818182  14.0             43.0   

     correct recalls scaled experiment  lists subject  total ints  \
0                 83.000000        FR1   25.0  R1022J        39.0   
1                 98.958333        FR1   24.0  R1032D        10.0   
2                 75.000000     catFR1   12.0  R1032D         4.0   
3                 93.055556     catFR1   18.0  R1032D         3.0   
4                 28.125000        FR1   24.0  R1049J        55.0   
..                      ...        ...    ...     ...         ...   
110              258.333333     catFR1   12.0  R1530J        28.0   
111              104.545455     catFR1   11.0  R1536J         6.0   
112               93.750000     catFR1   12.0  R1536J        15.0   
113              104.166667     catFR1   12.0  R1536J        21.0   
114               97.727273     catFR1   11.0  R1536J        14.0   

     total scaled  
0       39.000000  
1       10.416667  
2        8.333333  
3        4.166667  
4       57.291667  
..            ...  
110     58.333333  
111     13.636364  
112     31.250000  
113     43.750000  
114     31.818182  

[115 rows x 13 columns]

In [249]:
avg_df = pd.DataFrame([])
sub_df = df_filter['subject'].unique()
for sub in sub_df:
    trials = df_filter[df_filter['subject']==sub]
    df_mean = trials.mean(axis=0)
    avg_eli = df_mean['ELI scaled']
    avg_pli = df_mean['PLI scaled']
    avg_int = df_mean['total scaled']
    avg_cor = df_mean['correct recalls scaled']
    avg_df = avg_df.append({
        'subject':sub,
        'average ELIs':avg_eli,
        'average PLIs':avg_pli,
        'average ints': avg_int,
        'average correct recalls':avg_cor,
        'BAI':df_mean['BAI'],
        'BDI':df_mean['BDI']
        }, ignore_index = True)
avg_df

BAI   BDI  average ELIs  average PLIs  average correct recalls  \
0    3.0   1.0     25.000000     14.000000                83.000000   
1    3.0   6.0      3.356481      4.282407                89.004630   
2    7.0   6.0     36.458333     20.833333                28.125000   
3   20.0  30.0     12.666667      6.000000               120.333333   
4   11.0  13.0     36.805556     13.194444                36.111111   
5    0.0  11.0      2.083333      0.000000               110.416667   
6    9.0  19.0     55.732323     19.754638                63.563520   
7   20.0   6.0      5.851326      3.464962                82.714015   
8    1.0   6.0      2.125000      2.250000               171.625000   
9    1.0   1.0      1.906703      3.490489               101.158288   
10   4.0   5.0     11.895833      7.177083               114.510417   
11  15.0  18.0     17.261905     10.714286                76.190476   
12  10.0  16.0     26.086957     55.434783                29.347826   
13   4.0  41.0      7.956722      9.659738                57.004400   
14   5.0  16.0     10.680556     29.805556                79.166667   
15   3.0  13.0      7.898551     10.173913                88.739130   
16  35.0   8.0      7.423611     15.541667               105.236111   
17   0.0   1.0     56.250000     25.000000                72.916667   
18  10.0  10.0     14.816919     14.955808                86.060606   
19  29.0  26.0     28.840909     21.340909                61.083333   
20  10.0   1.0     32.500000     14.791667               110.625000   
21  10.0  11.0      1.780303      2.833333                94.878788   
22  14.0  29.0      6.865530      8.428030                95.407197   
23   6.0   5.0     18.750000     23.106061               107.244318   
24   4.0   4.0     11.742424     16.429924               154.971591   
25  24.0  39.0      6.500000      8.500000               103.500000   
26  13.0  14.0     11.458333      8.333333                85.416667   
27  15.0  36.0      7.638889     24.074074                32.754630   
28   4.0  25.0     18.607955      4.159903                32.886905   
29   2.0   4.0      6.944444      2.777778               103.472222   
30  13.0  10.0     34.077381     18.452381                73.363095   
31   6.0   3.0     18.087121     14.002525               149.046717   
32   5.0   6.0      5.000000     15.000000                80.000000   
33  18.0  16.0      6.944444     29.861111               240.972222   
34  24.0  17.0      6.344697     23.768939               100.047348   

    average ints subject  
0      39.000000  R1022J  
1       7.638889  R1032D  
2      57.291667  R1049J  
3      18.666667  R1051J  
4      50.000000  R1059J  
5       2.083333  R1081J  
6      75.486961  R1083J  
7       9.316288  R1093J  
8       4.375000  R1107J  
9       5.397192  R1108J  
10     19.072917  R1145J  
11     27.976190  R1150J  
12     81.521739  R1241J  
13     17.616460  R1266J  
14     40.486111  R1286J  
15     18.072464  R1317D  
16     22.965278  R1330D  
17     81.250000  R1343J  
18     29.772727  R1347D  
19     50.181818  R1367D  
20     47.291667  R1376D  
21      4.613636  R1384J  
22     15.293561  R1389J  
23     41.856061  R1398J  
24     28.172348  R1401J  
25     15.000000  R1413D  
26     19.791667  R1436J  
27     31.712963  R1450D  
28     22.767857  R1456D  
29      9.722222  R1482J  
30     52.529762  R1505J  
31     32.089646  R1525J  
32     20.000000  R1526J  
33     36.805556  R1530J  
34     30.113636  R1536J

In [258]:
xi = 'BAI'
yi = 'average correct recalls'

# x = df_filter[xi]
# y = df_filter[yi]

x = avg_df[xi]
y = avg_df[yi]

coef = np.polyfit(x, y, 1)
poly1d_fn = np.poly1d(coef)
grad, inter, r, p, std_err = stats.linregress(x, y)

# plt.figure(figsize=(8, 8))
# plt.scatter(x, y)
# plt.plot(x, poly1d_fn(x), '--k')
# max_x = round(x.max(axis = 0) + 5, 5)
# max_y = round(y.max(axis = 0) + 5, 5)
# plt.xticks(np.arange(0, max_x, 5))
# plt.yticks(np.arange(0, max_y, 10))
# plt.xlabel(xi)
# plt.ylabel(yi)
print('r = ',r, 'p = ', p)
print('y =', grad, '*x + ', inter)
fig, ax = plt.subplots()
sb.regplot(data = df_filter, x=x, y=y, ax=ax)
plt.title(xi + ' vs ' + yi)
# ax.set_xlim(0, 25)
# ax.set_xticks(range(0,25), 5)
#add error bars, combine FR/pyFR, catFR separate, check out Courier
#remove 5s and 4 intrusion criteria

r =  0.018899732285055 p =  0.9141852230107966
y = 0.09291311326872442 *x +  91.07517149539824


Text(0.5, 1.0, 'BAI vs average correct recalls')